# Module 09: Updates and Maintenance

**Difficulty**: ⭐⭐  
**Estimated Time**: 50 minutes  
**Prerequisites**: 
- Module 02 (npm) OR Module 03 (Docker) completed
- n8n installed and running
- Basic understanding of version control concepts

## Learning Objectives

By the end of this notebook, you will be able to:

1. Perform complete backups of workflows, configuration, and data
2. Update n8n safely using npm or Docker methods
3. Implement version pinning strategies for stability
4. Handle database migrations during updates
5. Execute rollback procedures when updates fail
6. Test updates in a safe environment before production

## Why Updates and Maintenance Matter

n8n releases updates frequently (typically 1-2 per month):

**Updates bring**:
- New integrations and nodes
- Bug fixes and stability improvements
- Security patches
- Performance enhancements
- New features

**But updates can also**:
- Introduce breaking changes
- Require database migrations
- Change node behavior
- Cause compatibility issues

### The Update Dilemma

```
Stay on old version:
  ✅ Stable and tested
  ❌ Missing new features
  ❌ Security vulnerabilities
  ❌ No bug fixes

Always update immediately:
  ✅ Latest features
  ✅ Security patches
  ❌ Risk of breaking changes
  ❌ Untested in your environment
```

**Solution**: Backup, test, update, verify.

### Update Frequency Recommendations

| Environment | Update Frequency | Testing Required |
|-------------|------------------|------------------|
| Development | Every release | Minimal |
| Personal Production | Monthly | Basic |
| Team Production | Quarterly | Thorough |
| Business Critical | As needed | Extensive |

## Part 1: Backup Procedures

**Golden rule**: ALWAYS backup before updates.

### What to Backup

1. **Workflows** (JSON exports)
2. **.n8n directory** (database, encryption key, credentials)
3. **Environment variables** (configuration)
4. **Docker volumes** (if using Docker)
5. **Startup scripts** (batch files, ecosystem.config.js)

### Method 1: Workflow Export (Essential)

#### Via n8n UI

1. Open n8n interface
2. Go to "Workflows"
3. Select all workflows (Ctrl+A)
4. Click "..." menu → Export
5. Save to `C:\n8n\backups\workflows-YYYY-MM-DD.json`

#### Via n8n CLI

```bash
# Export all workflows
npx n8n export:workflow --all --output=C:\n8n\backups\workflows\

# Export specific workflow
npx n8n export:workflow --id=5 --output=C:\n8n\backups\workflows\

# Export credentials (encrypted)
npx n8n export:credentials --all --output=C:\n8n\backups\credentials\
```

### Method 2: .n8n Directory Backup (Critical)

#### npm Installation

```powershell
# Stop n8n first
Stop-Process -Name "node" -Force

# Create backup directory with date
$backupDate = Get-Date -Format "yyyy-MM-dd_HHmmss"
$backupPath = "C:\n8n\backups\n8n-backup-$backupDate"

# Copy entire .n8n directory
Copy-Item -Path "$env:USERPROFILE\.n8n" -Destination $backupPath -Recurse

Write-Host "Backup completed: $backupPath"
```

#### Docker Installation

```powershell
# Create backup directory
$backupDate = Get-Date -Format "yyyy-MM-dd_HHmmss"
$backupFile = "C:\n8n\backups\n8n-docker-$backupDate.tar"

# Backup Docker volume
docker run --rm `
  -v n8n_data:/source:ro `
  -v C:\n8n\backups:/backup `
  ubuntu tar cvf /backup/n8n-docker-$backupDate.tar /source

Write-Host "Backup completed: $backupFile"
```

### Method 3: Environment Variables Backup

```powershell
# Export environment variables to file
$backupDate = Get-Date -Format "yyyy-MM-dd_HHmmss"
$envFile = "C:\n8n\backups\env-vars-$backupDate.txt"

# List all N8N environment variables
Get-ChildItem env: | Where-Object {$_.Name -like "N8N*"} | `
  ForEach-Object { "$($_.Name)=$($_.Value)" } | `
  Out-File -FilePath $envFile

# Also backup your startup script
Copy-Item "C:\n8n\start-n8n.bat" "C:\n8n\backups\start-n8n-$backupDate.bat"
```

### Method 4: Complete Backup Script

Create `C:\n8n\backup-n8n.ps1`:

```powershell
# Complete n8n Backup Script
param(
    [string]$BackupRoot = "C:\n8n\backups",
    [switch]$StopN8n = $false
)

$timestamp = Get-Date -Format "yyyy-MM-dd_HHmmss"
$backupDir = "$BackupRoot\backup-$timestamp"

Write-Host "Starting n8n backup: $timestamp" -ForegroundColor Green

# Create backup directory
New-Item -Path $backupDir -ItemType Directory -Force | Out-Null

# Stop n8n if requested
if ($StopN8n) {
    Write-Host "Stopping n8n..." -ForegroundColor Yellow
    Stop-Process -Name "node" -Force -ErrorAction SilentlyContinue
    Start-Sleep -Seconds 3
}

# Backup .n8n directory
Write-Host "Backing up .n8n directory..." -ForegroundColor Cyan
$n8nPath = "$env:USERPROFILE\.n8n"
if (Test-Path $n8nPath) {
    Copy-Item -Path $n8nPath -Destination "$backupDir\dot-n8n" -Recurse
    Write-Host "  ✅ .n8n directory backed up" -ForegroundColor Green
} else {
    Write-Host "  ⚠️  .n8n directory not found" -ForegroundColor Yellow
}

# Backup environment variables
Write-Host "Backing up environment variables..." -ForegroundColor Cyan
Get-ChildItem env: | Where-Object {$_.Name -like "N8N*" -or $_.Name -like "*NODE*"} | `
    ForEach-Object { "$($_.Name)=$($_.Value)" } | `
    Out-File -FilePath "$backupDir\environment-variables.txt"
Write-Host "  ✅ Environment variables backed up" -ForegroundColor Green

# Backup startup scripts
Write-Host "Backing up startup scripts..." -ForegroundColor Cyan
$scripts = @("C:\n8n\start-n8n.bat", "C:\n8n\ecosystem.config.js")
foreach ($script in $scripts) {
    if (Test-Path $script) {
        Copy-Item $script "$backupDir\"
        Write-Host "  ✅ Backed up: $(Split-Path $script -Leaf)" -ForegroundColor Green
    }
}

# Create backup manifest
$manifest = @"
n8n Backup Manifest
Created: $timestamp
Backup Location: $backupDir

Contents:
- .n8n directory (database, encryption keys, credentials)
- Environment variables
- Startup scripts

Restore Instructions:
1. Stop n8n
2. Copy dot-n8n/* to $env:USERPROFILE\.n8n\
3. Review environment-variables.txt and update your startup script
4. Start n8n
"@

$manifest | Out-File "$backupDir\MANIFEST.txt"

Write-Host ""
Write-Host "✅ Backup completed successfully!" -ForegroundColor Green
Write-Host "Location: $backupDir" -ForegroundColor Yellow
Write-Host "Size: $((Get-ChildItem $backupDir -Recurse | Measure-Object -Property Length -Sum).Sum / 1MB) MB" -ForegroundColor Yellow
```

Usage:
```powershell
# Backup without stopping n8n
.\backup-n8n.ps1

# Backup and stop n8n
.\backup-n8n.ps1 -StopN8n

# Backup to custom location
.\backup-n8n.ps1 -BackupRoot "D:\Backups\n8n"
```

In [ ]:
# Backup verification tool

import os
from pathlib import Path
import json
from datetime import datetime

def verify_backup(backup_path):
    """
    Verify that a backup contains all essential n8n data.
    
    Args:
        backup_path: Path to backup directory
    """
    print("=" * 70)
    print("N8N BACKUP VERIFICATION")
    print("=" * 70)
    
    backup_dir = Path(backup_path)
    
    if not backup_dir.exists():
        print(f"\n❌ Backup directory not found: {backup_path}")
        return False
    
    print(f"\n📂 Backup Location: {backup_path}")
    print(f"📅 Created: {datetime.fromtimestamp(backup_dir.stat().st_mtime)}")
    
    # Calculate total size
    total_size = sum(f.stat().st_size for f in backup_dir.rglob('*') if f.is_file())
    print(f"💾 Total Size: {total_size / (1024*1024):.2f} MB")
    
    print("\n" + "─" * 70)
    print("CHECKING BACKUP CONTENTS:")
    print("─" * 70)
    
    checks_passed = 0
    checks_total = 0
    
    # Check for .n8n directory
    checks_total += 1
    n8n_dir = backup_dir / "dot-n8n"
    if n8n_dir.exists():
        print("\n✅ .n8n directory found")
        checks_passed += 1
        
        # Check for database
        checks_total += 1
        db_file = n8n_dir / "database.sqlite"
        if db_file.exists():
            db_size = db_file.stat().st_size / (1024*1024)
            print(f"   ✅ Database file: {db_size:.2f} MB")
            checks_passed += 1
        else:
            print("   ❌ Database file missing!")
        
        # Check for config
        checks_total += 1
        config_dir = n8n_dir / "config"
        if config_dir.exists():
            print("   ✅ Config directory found")
            checks_passed += 1
        else:
            print("   ⚠️  Config directory missing (may be OK if first run)")
    else:
        print("\n❌ .n8n directory NOT found in backup")
    
    # Check for environment variables
    checks_total += 1
    env_file = backup_dir / "environment-variables.txt"
    if env_file.exists():
        lines = env_file.read_text().strip().split('\n')
        print(f"\n✅ Environment variables: {len(lines)} variables backed up")
        checks_passed += 1
    else:
        print("\n⚠️  Environment variables file not found")
    
    # Check for startup scripts
    checks_total += 1
    scripts_found = []
    for script in ["start-n8n.bat", "ecosystem.config.js"]:
        if (backup_dir / script).exists():
            scripts_found.append(script)
    
    if scripts_found:
        print(f"\n✅ Startup scripts: {', '.join(scripts_found)}")
        checks_passed += 1
    else:
        print("\n⚠️  No startup scripts found in backup")
    
    # Check for manifest
    manifest_file = backup_dir / "MANIFEST.txt"
    if manifest_file.exists():
        print("\n✅ Backup manifest found")
    
    # Summary
    print("\n" + "=" * 70)
    print("VERIFICATION SUMMARY")
    print("=" * 70)
    
    success_rate = (checks_passed / checks_total) * 100
    print(f"\nChecks passed: {checks_passed}/{checks_total} ({success_rate:.0f}%)")
    
    if checks_passed == checks_total:
        print("\n🎉 BACKUP IS COMPLETE AND VALID")
        print("\n✅ Safe to proceed with updates")
    elif checks_passed >= checks_total * 0.75:
        print("\n⚠️  BACKUP IS MOSTLY COMPLETE")
        print("\nMissing items should be addressed, but critical data is backed up")
    else:
        print("\n❌ BACKUP IS INCOMPLETE")
        print("\n⚠️  DO NOT update until backup is complete!")
        print("\nRun the backup script again and verify all components")
    
    print("\n" + "=" * 70)
    
    return checks_passed == checks_total

# Example usage
example_backup_path = "C:\\n8n\\backups\\backup-2024-01-15_120000"
print(f"To verify a backup, run:")
print(f'verify_backup("{example_backup_path}")')
print("\nReplace with your actual backup path")

## Part 2: Updating n8n (npm Method)

### Check Current Version

```powershell
# Check installed n8n version
n8n --version

# Check latest available version
npm view n8n version

# Check for outdated packages
npm outdated -g
```

### Update Procedure

#### Step 1: Backup (ALWAYS)

```powershell
# Run backup script
.\backup-n8n.ps1 -StopN8n
```

#### Step 2: Update n8n

```powershell
# Update to latest version
npm install -g n8n@latest

# OR update to specific version
npm install -g n8n@1.10.0
```

#### Step 3: Verify Update

```powershell
# Check new version
n8n --version
```

#### Step 4: Start n8n

```powershell
# Start n8n
n8n

# OR use your startup script
C:\n8n\start-n8n.bat
```

Watch for:
- Database migration messages
- Any error messages
- Successful startup confirmation

#### Step 5: Verify Functionality

1. Open http://localhost:5678
2. Check all workflows are visible
3. Execute a test workflow manually
4. Verify scheduled workflows trigger
5. Check credentials are accessible
6. Test webhook workflows

### Database Migrations

Some updates include database migrations:

```
Database migration needed. Running migration...
Migration successful!
```

**Important**: 
- Migrations are ONE-WAY operations
- Cannot downgrade without database rollback
- Always backup before migrations
- Let migrations complete (can take several minutes)

### Handling Update Failures

If update fails:

```powershell
# 1. Note the error message

# 2. Stop n8n
# (Ctrl+C or stop process)

# 3. Check npm cache
npm cache clean --force

# 4. Try update again
npm install -g n8n@latest

# 5. If still fails, reinstall
npm uninstall -g n8n
npm install -g n8n
```

If database migration fails:

```powershell
# Revert database migration
n8n db:revert

# This undoes the last migration
# Then restore from backup
```

## Part 3: Updating n8n (Docker Method)

### Check Current Version

```powershell
# Check container version
docker exec n8n n8n --version

# Check image version
docker image inspect docker.n8n.io/n8nio/n8n:latest | findstr "Created"
```

### Update Procedure

#### Method 1: Using docker run

```powershell
# 1. Backup Docker volume
$backupDate = Get-Date -Format "yyyy-MM-dd_HHmmss"
docker run --rm `
  -v n8n_data:/source:ro `
  -v C:\n8n\backups:/backup `
  ubuntu tar cvf /backup/n8n-before-update-$backupDate.tar /source

# 2. Pull new image
docker pull docker.n8n.io/n8nio/n8n:latest

# 3. Stop and remove old container
docker stop n8n
docker rm n8n

# 4. Start new container with same configuration
docker run -d `
  --name n8n `
  -p 5678:5678 `
  -e N8N_BASIC_AUTH_ACTIVE=true `
  -e N8N_BASIC_AUTH_USER=admin `
  -e N8N_BASIC_AUTH_PASSWORD=your_password `
  -v n8n_data:/home/node/.n8n `
  --restart unless-stopped `
  docker.n8n.io/n8nio/n8n:latest

# 5. Verify
docker logs n8n --tail 50
```

#### Method 2: Using Docker Compose (Recommended)

```powershell
# 1. Navigate to docker-compose directory
cd C:\n8n-docker

# 2. Backup volume
docker compose exec n8n n8n export:workflow --all --output=/home/node/.n8n/backups/

# 3. Pull new image
docker compose pull

# 4. Stop and recreate containers
docker compose down
docker compose up -d

# 5. View logs
docker compose logs -f n8n
```

### Version Pinning

To use specific version instead of `latest`:

**docker-compose.yml**:
```yaml
services:
  n8n:
    image: docker.n8n.io/n8nio/n8n:1.10.0  # ← Specific version
    # ... rest of configuration
```

**docker run**:
```powershell
docker pull docker.n8n.io/n8nio/n8n:1.10.0
docker run ... docker.n8n.io/n8nio/n8n:1.10.0
```

### Rollback Procedure (Docker)

```powershell
# 1. Stop current container
docker stop n8n
docker rm n8n

# 2. Restore backup
docker run --rm `
  -v n8n_data:/target `
  -v C:\n8n\backups:/backup `
  ubuntu bash -c "rm -rf /target/* && tar xvf /backup/n8n-before-update-*.tar -C /"

# 3. Start with previous version
docker run ... docker.n8n.io/n8nio/n8n:1.9.0  # Previous version
```

## Part 4: Version Pinning Strategies

### Why Pin Versions?

**Advantages**:
- Predictable behavior
- No surprise breaking changes
- Controlled update schedule
- Easier troubleshooting (known version)

**Disadvantages**:
- Miss security patches
- No automatic bug fixes
- Manual update process
- May fall behind on features

### Pinning Strategies by Environment

#### Development

```
Strategy: Always latest
Reason: Test new features, find bugs early

npm: npm install -g n8n@latest
Docker: image: docker.n8n.io/n8nio/n8n:latest
```

#### Personal Production

```
Strategy: Pin major.minor, auto-update patch
Reason: Get bug fixes, avoid breaking changes

npm: npm install -g n8n@1.10.x
Docker: image: docker.n8n.io/n8nio/n8n:1.10
```

#### Team/Business

```
Strategy: Pin exact version
Reason: Maximum stability, controlled updates

npm: npm install -g n8n@1.10.3
Docker: image: docker.n8n.io/n8nio/n8n:1.10.3
```

### Update Schedule

**Recommended approach**:

1. **Monitor releases**: Watch n8n GitHub releases
2. **Read changelog**: Understand what changed
3. **Test in development**: Update dev environment first
4. **Wait 1-2 weeks**: Let community find bugs
5. **Schedule update**: Plan maintenance window
6. **Backup**: Always backup before update
7. **Update production**: With rollback plan ready
8. **Monitor**: Watch for issues post-update

### Semantic Versioning Guide

n8n follows semantic versioning (semver):

```
Version Format: MAJOR.MINOR.PATCH
Example: 1.10.3

MAJOR (1.x.x → 2.x.x):
  - Breaking changes
  - May require workflow updates
  - Test thoroughly before updating

MINOR (1.10.x → 1.11.x):
  - New features
  - New nodes
  - Generally backward compatible
  - Test before production

PATCH (1.10.3 → 1.10.4):
  - Bug fixes
  - Security patches
  - Safe to update quickly
  - Minimal testing needed
```

In [ ]:
# Version comparison and update advisor

def compare_versions(current, latest):
    """
    Compare n8n versions and provide update recommendations.
    
    Args:
        current: Current installed version (e.g., "1.9.5")
        latest: Latest available version (e.g., "1.10.2")
    """
    print("=" * 70)
    print("N8N VERSION UPDATE ADVISOR")
    print("=" * 70)
    
    # Parse versions
    try:
        curr_parts = [int(x) for x in current.split('.')]
        latest_parts = [int(x) for x in latest.split('.')]
    except ValueError:
        print("\n❌ Invalid version format. Use MAJOR.MINOR.PATCH (e.g., 1.10.3)")
        return
    
    curr_major, curr_minor, curr_patch = curr_parts
    latest_major, latest_minor, latest_patch = latest_parts
    
    print(f"\n📦 Current Version: {current}")
    print(f"📦 Latest Version:  {latest}")
    
    # Check if already latest
    if current == latest:
        print("\n✅ You are running the latest version!")
        print("\nNo update needed.")
        return
    
    # Check if somehow ahead (development/beta)
    if (curr_major > latest_major or 
        (curr_major == latest_major and curr_minor > latest_minor) or
        (curr_major == latest_major and curr_minor == latest_minor and curr_patch > latest_patch)):
        print("\n⚠️  You are running a newer version than latest stable")
        print("\nYou may be on a beta or development version.")
        return
    
    # Determine update type
    print("\n" + "─" * 70)
    print("UPDATE ANALYSIS")
    print("─" * 70)
    
    if latest_major > curr_major:
        update_type = "MAJOR"
        risk = "HIGH"
        color = "🔴"
    elif latest_minor > curr_minor:
        update_type = "MINOR"
        risk = "MEDIUM"
        color = "🟡"
    else:
        update_type = "PATCH"
        risk = "LOW"
        color = "🟢"
    
    versions_behind = {
        'major': latest_major - curr_major,
        'minor': latest_minor - curr_minor if latest_major == curr_major else latest_minor,
        'patch': latest_patch - curr_patch if latest_major == curr_major and latest_minor == curr_minor else latest_patch
    }
    
    print(f"\n{color} Update Type: {update_type}")
    print(f"⚠️  Risk Level: {risk}")
    
    if versions_behind['major'] > 0:
        print(f"\nYou are {versions_behind['major']} major version(s) behind")
    if versions_behind['minor'] > 0:
        print(f"You are {versions_behind['minor']} minor version(s) behind")
    if versions_behind['patch'] > 0:
        print(f"You are {versions_behind['patch']} patch version(s) behind")
    
    # Provide recommendations
    print("\n" + "─" * 70)
    print("RECOMMENDATIONS")
    print("─" * 70)
    
    if update_type == "MAJOR":
        print("\n🔴 MAJOR UPDATE - PROCEED WITH CAUTION")
        print("\n⚠️  This update may include breaking changes")
        print("\nBefore updating:")
        print("  1. ✅ Read release notes thoroughly")
        print("  2. ✅ Backup everything (workflows, database, config)")
        print("  3. ✅ Test in development environment first")
        print("  4. ✅ Review all workflows for compatibility")
        print("  5. ✅ Plan rollback procedure")
        print("  6. ✅ Schedule maintenance window")
        print("\nExpected impact:")
        print("  • Workflows may need updates")
        print("  • Node behavior may change")
        print("  • Database migration required")
        print("  • Downtime: 15-60 minutes")
        
    elif update_type == "MINOR":
        print("\n🟡 MINOR UPDATE - STANDARD CAUTION")
        print("\n✅ Generally safe, but test first")
        print("\nBefore updating:")
        print("  1. ✅ Backup workflows and database")
        print("  2. ✅ Review release notes")
        print("  3. ✅ Test critical workflows after update")
        print("  4. ⚠️  Optional: Test in dev environment")
        print("\nExpected impact:")
        print("  • New features available")
        print("  • Existing workflows should work")
        print("  • Possible database migration")
        print("  • Downtime: 5-15 minutes")
        
    else:  # PATCH
        print("\n🟢 PATCH UPDATE - LOW RISK")
        print("\n✅ Safe to update, minimal testing needed")
        print("\nBefore updating:")
        print("  1. ✅ Backup workflows (quick export)")
        print("  2. ⚠️  Optional: Full backup")
        print("  3. ⚠️  Optional: Read release notes")
        print("\nExpected impact:")
        print("  • Bug fixes and security patches")
        print("  • No workflow changes needed")
        print("  • Unlikely to have database migration")
        print("  • Downtime: 2-5 minutes")
    
    # Update commands
    print("\n" + "─" * 70)
    print("UPDATE COMMANDS")
    print("─" * 70)
    
    print("\nnpm installation:")
    print(f"  npm install -g n8n@{latest}")
    
    print("\nDocker installation:")
    print(f"  docker pull docker.n8n.io/n8nio/n8n:{latest}")
    print("  # Then recreate container with new image")
    
    print("\n" + "=" * 70)

# Example comparisons
test_scenarios = [
    ("1.9.5", "1.10.2"),  # Minor update
    ("1.10.0", "1.10.3"), # Patch update
    ("0.235.0", "1.0.0"), # Major update
]

for current, latest in test_scenarios:
    compare_versions(current, latest)
    print("\n" + "="*70 + "\n")

## Part 5: Rollback Procedures

Sometimes updates go wrong. Here's how to rollback.

### When to Rollback

Rollback if you experience:
- Workflows fail to execute
- n8n won't start after update
- Database migration fails
- Critical functionality broken
- Performance significantly degraded

### npm Rollback

#### Option 1: Install Previous Version

```powershell
# Stop n8n
Stop-Process -Name "node" -Force

# Install previous version
npm install -g n8n@1.9.5  # Replace with your previous version

# If database was migrated, revert migration
n8n db:revert

# Restore backup if needed
# 1. Delete current .n8n directory
# 2. Copy backup to $env:USERPROFILE\.n8n

# Start n8n
n8n
```

#### Option 2: Complete Restore from Backup

```powershell
# 1. Stop n8n
Stop-Process -Name "node" -Force

# 2. Remove current .n8n directory
Remove-Item -Path "$env:USERPROFILE\.n8n" -Recurse -Force

# 3. Restore backup
$backupDir = "C:\n8n\backups\backup-2024-01-15_120000\dot-n8n"
Copy-Item -Path $backupDir -Destination "$env:USERPROFILE\.n8n" -Recurse

# 4. Install previous n8n version
npm install -g n8n@1.9.5

# 5. Start n8n
n8n
```

### Docker Rollback

#### Option 1: Use Previous Image

```powershell
# 1. Stop current container
docker stop n8n
docker rm n8n

# 2. Start with previous version
docker run -d `
  --name n8n `
  -p 5678:5678 `
  -e N8N_BASIC_AUTH_ACTIVE=true `
  -e N8N_BASIC_AUTH_USER=admin `
  -e N8N_BASIC_AUTH_PASSWORD=your_password `
  -v n8n_data:/home/node/.n8n `
  --restart unless-stopped `
  docker.n8n.io/n8nio/n8n:1.9.5  # Previous version

# 3. Check logs
docker logs n8n -f
```

#### Option 2: Restore Volume from Backup

```powershell
# 1. Stop and remove container
docker stop n8n
docker rm n8n

# 2. Delete current volume data
docker run --rm -v n8n_data:/data ubuntu rm -rf /data/*

# 3. Restore from backup
docker run --rm `
  -v n8n_data:/data `
  -v C:\n8n\backups:/backup `
  ubuntu tar xvf /backup/n8n-before-update-*.tar -C /

# 4. Start with previous version
docker run ... docker.n8n.io/n8nio/n8n:1.9.5
```

### Database Migration Rollback

If migration occurred but failed or caused issues:

```bash
# Revert last migration
n8n db:revert

# Revert multiple migrations
n8n db:revert
n8n db:revert
# Repeat for each migration to revert
```

**Warning**: Only works if:
- Migration completed but caused problems
- You haven't run workflows that modified the new schema
- Database is in consistent state

If migration failed mid-way, restore from backup instead.

## Exercises

### Exercise 1: Complete Backup Procedure (Easy)

**Task**: Create a complete backup of your n8n installation.

**Steps**:
1. Create backup directory structure at `C:\n8n\backups`
2. Export all workflows via n8n UI
3. Stop n8n
4. Copy entire .n8n directory to backup location
5. Document all environment variables
6. Copy startup scripts
7. Create backup manifest file
8. Restart n8n
9. Verify backup completeness using verification tool
10. Test restore process (optional but recommended)

**Verification**: Backup directory contains all essential files, verification tool shows 100% complete.

**Deliverable**: Complete backup with manifest file and verification results.

### Exercise 2: Safe Update Process (Medium)

**Task**: Update n8n to latest version following best practices.

**Steps**:
1. Check current version: `n8n --version`
2. Check latest version: `npm view n8n version`
3. Read release notes for new version
4. Perform complete backup (Exercise 1)
5. Stop n8n
6. Update n8n:
   - npm: `npm install -g n8n@latest`
   - Docker: Pull new image and recreate container
7. Start n8n and watch for migration messages
8. Verify new version: `n8n --version`
9. Test critical workflows
10. Monitor for issues for 24 hours

**Verification**: n8n runs on new version, all workflows function correctly, no errors in logs.

**Deliverable**: Update log with version numbers, migration messages, and test results.

### Exercise 3: Rollback Simulation (Hard)

**Task**: Practice complete update and rollback cycle.

**Steps**:
1. Document current state:
   - Version number
   - All workflows
   - Configuration
2. Create comprehensive backup
3. Update to latest version
4. Verify update successful
5. Intentionally trigger rollback need:
   - OR: Wait for actual issue
   - OR: Simulate by testing rollback procedure
6. Execute complete rollback:
   - Restore database
   - Revert to previous version
   - Verify workflows work
7. Document entire process
8. Create rollback runbook for future use
9. Measure downtime during rollback
10. Identify improvement opportunities

**Verification**: Successfully rollback to previous version with all workflows functioning, complete documentation.

**Deliverable**: Detailed rollback runbook with timings, screenshots, and lessons learned.

## Summary

In this module, you learned comprehensive update and maintenance procedures for n8n:

### Backup Procedures
1. **Workflow export**: JSON files via UI or CLI
2. **.n8n directory**: Database, encryption keys, credentials
3. **Environment variables**: Configuration documentation
4. **Docker volumes**: Container data persistence
5. **Automated backups**: PowerShell scripts for consistency

### Update Methods
6. **npm updates**: `npm install -g n8n@latest`
7. **Docker updates**: Pull new image, recreate container
8. **Version checking**: Know current vs available versions
9. **Migration handling**: Let database migrations complete
10. **Post-update verification**: Test workflows after updating

### Version Management
11. **Semantic versioning**: Understanding MAJOR.MINOR.PATCH
12. **Version pinning**: Control when updates occur
13. **Update strategies**: Different approaches by environment
14. **Release monitoring**: Stay informed about updates
15. **Changelog review**: Understand changes before updating

### Rollback Procedures
16. **Database revert**: `n8n db:revert` for migrations
17. **Version rollback**: Install previous npm or Docker version
18. **Data restore**: Copy backup to .n8n directory
19. **Volume restore**: Docker backup restoration
20. **Verification**: Ensure rollback successful

### Key Takeaways

✅ **ALWAYS backup before updates** - No exceptions  
✅ **Read release notes** - Understand what's changing  
✅ **Test in development** - Find issues before production  
✅ **Pin versions for stability** - Control your update schedule  
✅ **Monitor after updates** - Watch for issues  
✅ **Have rollback plan** - Be ready to revert  

### Update Decision Matrix

| Update Type | Risk | Test Required | Backup | Rollback Plan |
|-------------|------|---------------|--------|---------------|
| Patch (x.x.3→x.x.4) | Low | Minimal | Quick | Optional |
| Minor (x.10→x.11) | Medium | Standard | Full | Ready |
| Major (1.x→2.x) | High | Extensive | Full + Test | Required |

### Maintenance Schedule

**Weekly**:
- Monitor n8n logs for errors
- Check execution history for failed workflows
- Verify auto-start still working

**Monthly**:
- Export workflow backups
- Review available updates
- Check disk space usage
- Test backup restoration

**Quarterly**:
- Update to latest stable version
- Full backup with verification
- Review and clean up old workflows
- Security audit

**Annually**:
- Complete system review
- Evaluate laptop vs server deployment
- Review all credentials and rotate
- Update documentation

### What's Next

- **Module 10**: Production deployment strategies (final module)
  - Laptop vs server decision
  - Migration planning
  - Cost analysis
  - When to upgrade infrastructure

### Additional Resources

- [n8n Release Notes](https://github.com/n8n-io/n8n/releases)
- [n8n Update Guide](https://docs.n8n.io/hosting/installation/updating/)
- [Database Migration Documentation](https://docs.n8n.io/hosting/configuration/database/)
- [Backup Best Practices](https://docs.n8n.io/hosting/installation/server-setups/)

### Remember

Updates bring new features and fixes, but they also introduce risk. The key to successful maintenance is preparation: good backups, thorough testing, and clear rollback procedures. Spend 30 minutes on proper backup and update procedures to avoid hours of downtime from failed updates. Your future self will thank you when that rollback plan saves your production environment.